Generate synthetic libraries using NK model

In [4]:
import torch
from torch import distributions as dist
import itertools

import operator
import pickle
import importlib
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')
sns.set_context('paper')
# Plot adjustments:
plt.rcParams.update({'ytick.labelsize': 15})
plt.rcParams.update({'xtick.labelsize': 15})
plt.rcParams.update({'axes.labelsize': 35})
plt.rcParams.update({'legend.fontsize': 30})
plt.rcParams.update({'axes.titlesize': 16})

from gptorch import kernels, models
import bases, helpers, opt

In [5]:
class NK_Model(object):
    """ A Kauffman NK Model, with tuneable ruggedness K. 
    
    Adapted from: https://github.com/dmasad/NK-Model-Experiments/blob/master/NKModel.py"""
        
    ## is A needed as well???
    def __init__(self, n, k=2, random_fitness=True, fitness_vector = None):
        """ Initialize a NK Model."""
        self.n = n
        self.k = k
        self.random_fitness = random_fitness
        self.graph = None
        if self.random_fitness:
            # Build a random fitness vector of each offset's contribution.
            self.fitness_vector = [random.randrange(-5, 5) for i in range(self.k)]
        elif len(fitness_vector) == self.k:
            self.fitness_vector = fitness_vector
        else:
            raise Exception("Fitness vector wrong length!")


    # ======================================
    #         Fitness evaluation
    # ======================================
    
    def _local_fitness(self, aa):
        '''
        Evaluate the local fitness of an amino acid at a specific site.
        '''
        if len(aa) != self.k:
            raise Exception("Wrong amino acid length.")

        fitness = 0
        for i in range(self.k):
            fitness += aa[i] * self.fitness_vector[i]
        return fitness
    
    def eval_fitness(self, seq):
        '''
        Evaluate the fitness of an entire amino acid sequence.
        '''
        if len(seq) != self.n:
            raise Exception("Wrong length!")

        fitness = 0
        for i in range(self.n):
            aa = []
            for d in range(i-(self.k-1), i+1): # Loop from i-(k-1) to i, inclusive
                aa.append(seq[d])
            fitness += self._local_fitness(aa)
        return fitness

    
    # ======================================
    #         Optimization
    # ======================================

    def _num_to_tuple(self, num):
        '''
        Convert a number to a tuple of its binary representation.
        '''
        binary = bin(num)[2:]
        while len(binary) < self.n:
            binary = "0" + binary
        if len(binary) > self.n:
            raise Exception("Number too big to belong here!")
        binary = list(binary)
        bits = [int(b) for b in binary]
        return tuple(bits)
        
    def enumerate_space(self):
        '''
        Exhaustively enumerate the fitness of each point in the model space.
        '''
        model_space = {}
        for i in range(2**self.n):
            # Convert number to bit string
            seq = self._num_to_tuple(i)
            model_space[seq] = self.eval_fitness(seq)
        return model_space

In [6]:
nk = NK_Model(20)

In [8]:
nk.enumerate_space()

{(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 0,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1): 3,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0): 3,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1): 6,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0): 3,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1): 6,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0): 6,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1): 9,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0): 3,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1): 6,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0): 6,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1): 9,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0): 6,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1): 9,
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0): 9,
 (0, 0, 0,